In [3]:
!pip install -U langchain_aws

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.5
    Uninstalling langchain-core-0.2.5:
      Successfully uninstalled langchain-core-0.2.5
  Attempting uninstall: langchain_aws
    Found existing installation: langchain-aws 0.1.6
    Uninstalling langchain-aws-0.1.6:
      Successfully uninstalled langchain-aws-0.1.6


In [1]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [19]:
import boto3
import json
import random

from botocore.config import Config
config = Config(
    # region_name=random.choice(['us-west-2','us-east-1']),
    # region_name='us-east-1',
    read_timeout=600, ## Timeout 시간 조정
    retries = dict(
        max_attempts = 8 ## Retry 횟수 조정
    )
)

from langchain_aws import ChatBedrockConverse
## https://api.python.langchain.com/en/latest/chat_models/langchain_aws.chat_models.bedrock.ChatBedrock.html

llm = ChatBedrockConverse(
    region_name=random.choice(['us-west-2','us-east-1']),
    model_id="anthropic.claude-3-sonnet-20240229-v1:0", #파운데이션 모델 지정
    max_tokens=1024,
    temperature=0,
    config=config
) #Claude 속성 구성

In [20]:
embeddings = BedrockEmbeddings() #Titan 임베딩 클라이언트를 생성합니다.

pdf_path = "2022-Shareholder-Letter.pdf" #이 이름을 가진 로컬 PDF 파일을 가정합니다.

loader = PyPDFLoader(file_path=pdf_path) #PDF 파일 로드하기

text_splitter = RecursiveCharacterTextSplitter( #텍스트 분할기 만들기
    separators=["\n\n", "\n", ".", " "], #(1) 단락, (2) 줄, (3) 문장 또는 (4) 단어 순서로 청크를 분할합니다.
    chunk_size=1000, #위의 구분 기호를 사용하여 1000자 청크로 나눕니다.
    chunk_overlap=100 #이전 청크와 겹칠 수 있는 문자 수입니다.
)

index_creator = VectorstoreIndexCreator( #벡터 스토어 팩토리 만들기
    vectorstore_cls=FAISS, #데모 목적으로 인메모리 벡터 저장소를 사용합니다.
    embedding=embeddings, #Titan 임베딩 사용
    text_splitter=text_splitter, #재귀적 텍스트 분할기 사용하기
)

index_from_loader = index_creator.from_loaders([loader]) #로드된 PDF에서 벡터 스토어 인덱스를 생성합니다.

In [21]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    human_prefix = 'User', 
    ai_prefix='Assistant', 
    memory_key="chat_history", 
    return_messages=True
) #이전 메시지의 기록을 유지합니다.

In [22]:
input_text = '회사의 Generative AI 전략은 무엇인가요?'

In [23]:
from langchain.chains import ConversationalRetrievalChain

conversation_with_retrieval = ConversationalRetrievalChain.from_llm(
    llm, 
    index_from_loader.vectorstore.as_retriever(), 
    memory=memory
)
chat_response = conversation_with_retrieval.invoke({"question": input_text}) #사용자 메시지, 기록 및 지식을 모델에 전달합니다.

chat_response['answer']

'아마존은 Generative AI, 특히 대규모 언어 모델(LLM)에 상당한 투자를 하고 있습니다. 주요 전략은 다음과 같습니다:\n\n1. 자체 LLM 개발 및 모든 고객 경험 개선을 위한 활용\n아마존은 자체 LLM을 개발 중이며, 이를 통해 소비자, 판매자, 브랜드, 크리에이터 등 모든 고객 경험을 혁신하고자 합니다.\n\n2. AWS를 통한 Generative AI 민주화\nAWS는 Trainium, Inferentia 등 가격 대비 성능이 우수한 ML 칩을 제공하여 기업들이 LLM을 저렴하게 학습하고 운영할 수 있도록 지원합니다. 또한 CodeWhisperer 등 Generative AI 애플리케이션도 제공합니다.\n\n3. 광고 및 추천 시스템 고도화\n아마존은 LLM과 머신러닝을 활용하여 광고 및 제품 추천 알고리즘을 지속적으로 개선하고 있습니다.\n\n요약하면 아마존은 자체 LLM 개발, AWS 클라우드 서비스를 통한 Generative AI 민주화, 그리고 자사 서비스의 광고/추천 고도화를 위해 Generative AI에 적극 투자하고 있습니다.'

In [24]:
input_text = 'LLM 기반 광고/추천 시스템 고도화에 대해 좀 더 자세히 알려줄 수 있어?'

In [25]:
conversation_with_retrieval = ConversationalRetrievalChain.from_llm(
    llm, 
    index_from_loader.vectorstore.as_retriever(), 
    memory=memory
)
chat_response = conversation_with_retrieval.invoke({"question": input_text}) #사용자 메시지, 기록 및 지식을 모델에 전달합니다.

chat_response['answer']

'제프 베조스의 연례 주주 서한에서 언급된 내용을 종합해보면, 아마존은 LLM(Large Language Model)과 생성형 AI 기술을 활용하여 광고 및 제품 추천 시스템을 고도화할 계획인 것으로 보입니다.\n\n구체적으로는 다음과 같은 방향으로 진행될 것으로 예상됩니다:\n\n1. LLM을 활용하여 고객의 검색 의도와 쇼핑 행동을 더 잘 이해하고, 이를 바탕으로 관련성 높은 광고와 제품을 추천하는 기능 개선\n\n2. 생성형 AI 모델을 활용하여 고객, 판매자, 브랜드, 크리에이터 등 다양한 이해관계자를 위한 맞춤형 경험 제공\n\n3. AWS 클라우드 상에서 LLM 및 생성형 AI 모델을 효율적으로 학습하고 운영할 수 있는 인프라 및 서비스 제공\n\n4. 개발자 생산성 향상을 위한 코드 제안 등 LLM 기반 개발자 도구 활용\n\n아마존은 이러한 LLM과 생성형 AI 기술 활용을 통해 고객 경험 혁신, 광고 효율 제고, 개발자 생산성 향상 등의 효과를 기대하고 있는 것으로 보입니다.'

In [26]:
memory

ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='회사의 Generative AI 전략은 무엇인가요?'), AIMessage(content='아마존은 Generative AI, 특히 대규모 언어 모델(LLM)에 상당한 투자를 하고 있습니다. 주요 전략은 다음과 같습니다:\n\n1. 자체 LLM 개발 및 모든 고객 경험 개선을 위한 활용\n아마존은 자체 LLM을 개발 중이며, 이를 통해 소비자, 판매자, 브랜드, 크리에이터 등 모든 고객 경험을 혁신하고자 합니다.\n\n2. AWS를 통한 Generative AI 민주화\nAWS는 Trainium, Inferentia 등 가격 대비 성능이 우수한 ML 칩을 제공하여 기업들이 LLM을 저렴하게 학습하고 운영할 수 있도록 지원합니다. 또한 CodeWhisperer 등 Generative AI 애플리케이션도 제공합니다.\n\n3. 광고 및 추천 시스템 고도화\n아마존은 LLM과 머신러닝을 활용하여 광고 및 제품 추천 알고리즘을 지속적으로 개선하고 있습니다.\n\n요약하면 아마존은 자체 LLM 개발, AWS 클라우드 서비스를 통한 Generative AI 민주화, 그리고 자사 서비스의 광고/추천 고도화를 위해 Generative AI에 적극 투자하고 있습니다.'), HumanMessage(content='LLM 기반 광고/추천 시스템 고도화에 대해 좀 더 자세히 알려줄 수 있어?'), AIMessage(content='제프 베조스의 연례 주주 서한에서 언급된 내용을 종합해보면, 아마존은 LLM(Large Language Model)과 생성형 AI 기술을 활용하여 광고 및 제품 추천 시스템을 고도화할 계획인 것으로 보입니다.\n\n구체적으로는 다음과 같은 방향으로 진행될 것으로 예상됩니다:\n\n1. LLM을 활용하여 고객의 검색 의도와 쇼핑 행동을 더 잘 이해하고, 이를 바탕으로 관련성 높

## ConversationSummaryMemory

In [27]:
from langchain.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory(
    llm=llm,
    human_prefix = 'User', 
    ai_prefix='Assistant', 
    memory_key='chat_history'
)

In [28]:
from langchain.chains import ConversationalRetrievalChain

conversation_with_retrieval = ConversationalRetrievalChain.from_llm(
    llm, 
    index_from_loader.vectorstore.as_retriever(), 
    memory=summary_memory,
)
chat_response = conversation_with_retrieval.invoke({"question": input_text}) #사용자 메시지, 기록 및 지식을 모델에 전달합니다.

chat_response['answer']

'제프 베조스의 연례 주주 서한에서 언급된 내용을 종합해보면, 아마존은 LLM(Large Language Model)과 생성형 AI 기술을 활용하여 광고 및 제품 추천 시스템을 고도화할 계획인 것으로 보입니다.\n\n구체적으로는 다음과 같은 방향으로 진행될 것으로 예상됩니다:\n\n1. LLM을 활용하여 고객 경험 전반을 개선\n- 검색, 제품 추천, 맞춤형 콘텐츠 생성 등에 LLM 기술 적용\n\n2. 광고 선정 알고리즘 개선\n- 기계학습 기술로 광고 관련성을 높여 고객과 브랜드 모두에게 유용한 광고 제공\n\n3. 개발자 생산성 향상\n- AWS CodeWhisperer 등 LLM 기반 코드 자동완성 기능 제공\n\n4. 기업 고객을 위한 LLM 서비스 제공\n- AWS 클라우드 상에서 LLM 모델 학습/추론 환경 제공\n\n아마존은 LLM과 생성형 AI가 미래 혁신의 핵심 기술이 될 것으로 내다보고 있으며, 이를 통해 고객 경험 혁신과 새로운 수익원 창출을 모두 노리는 것으로 판단됩니다.'

## condense_question_prompt + chain_type

In [29]:
from langchain.prompts import PromptTemplate

# Define your system instruction
system_instruction = "The assistant should provide detailed explanations."

# Define your template with the system instruction
template = (
    f"{system_instruction} "
    "Combine the chat history and follow up question into "
    "a standalone question. Chat History: {chat_history}"
    "Follow up question: {question}"
)

# Create the prompt template
condense_question_prompt = PromptTemplate.from_template(template)

In [30]:
conversation_with_retrieval = ConversationalRetrievalChain.from_llm(
    llm, 
    index_from_loader.vectorstore.as_retriever(), 
    memory=memory,
    condense_question_prompt=condense_question_prompt,
    chain_type="stuff"
)
chat_response = conversation_with_retrieval.invoke({"question": input_text}) #사용자 메시지, 기록 및 지식을 모델에 전달합니다.

chat_response['answer']

'아마존이 LLM 기술을 광고 및 제품 추천 시스템에 적용하여 다음과 같은 방식으로 고도화할 것이라는 점을 잘 설명해주셨습니다:\n\n1. 고객 행동 및 의도 이해 개선\n2. 동적 맞춤형 광고/추천 문구 생성 \n3. 이미지, 비디오 등 멀티모달 데이터 활용\n4. 지속적인 데이터 학습을 통한 모델 성능 향상\n\nLLM의 자연어 처리, 텍스트 생성, 멀티모달 데이터 처리 능력을 활용하여 고객 맞춤형 광고/추천의 관련성과 정확도를 크게 높일 수 있을 것으로 보입니다. 아마존의 방대한 고객 데이터와 LLM 기술을 결합하면 매우 강력한 개인화 시스템을 구축할 수 있을 것 같습니다. 지속적인 모델 개선으로 서비스 품질도 꾸준히 향상될 것으로 기대됩니다.'

In [31]:
memory

ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='회사의 Generative AI 전략은 무엇인가요?'), AIMessage(content='아마존은 Generative AI, 특히 대규모 언어 모델(LLM)에 상당한 투자를 하고 있습니다. 주요 전략은 다음과 같습니다:\n\n1. 자체 LLM 개발 및 모든 고객 경험 개선을 위한 활용\n아마존은 자체 LLM을 개발 중이며, 이를 통해 소비자, 판매자, 브랜드, 크리에이터 등 모든 고객 경험을 혁신하고자 합니다.\n\n2. AWS를 통한 Generative AI 민주화\nAWS는 Trainium, Inferentia 등 가격 대비 성능이 우수한 ML 칩을 제공하여 기업들이 LLM을 저렴하게 학습하고 운영할 수 있도록 지원합니다. 또한 CodeWhisperer 등 Generative AI 애플리케이션도 제공합니다.\n\n3. 광고 및 추천 시스템 고도화\n아마존은 LLM과 머신러닝을 활용하여 광고 및 제품 추천 알고리즘을 지속적으로 개선하고 있습니다.\n\n요약하면 아마존은 자체 LLM 개발, AWS 클라우드 서비스를 통한 Generative AI 민주화, 그리고 자사 서비스의 광고/추천 고도화를 위해 Generative AI에 적극 투자하고 있습니다.'), HumanMessage(content='LLM 기반 광고/추천 시스템 고도화에 대해 좀 더 자세히 알려줄 수 있어?'), AIMessage(content='제프 베조스의 연례 주주 서한에서 언급된 내용을 종합해보면, 아마존은 LLM(Large Language Model)과 생성형 AI 기술을 활용하여 광고 및 제품 추천 시스템을 고도화할 계획인 것으로 보입니다.\n\n구체적으로는 다음과 같은 방향으로 진행될 것으로 예상됩니다:\n\n1. LLM을 활용하여 고객의 검색 의도와 쇼핑 행동을 더 잘 이해하고, 이를 바탕으로 관련성 높